In [1]:
import json

def load_json_dataset(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
valid_dataset_json_path = './poquad/poquad-dev.json'
train_dataset_json_path = './poquad/poquad-train.json'

valid_dataset_json = load_json_dataset(valid_dataset_json_path)
train_dataset_json = load_json_dataset(train_dataset_json_path)

In [4]:
print(len(valid_dataset_json['data']))
valid_dataset_json['data'][0]

1402


{'id': 9773,
 'title': 'Miszna',
 'summary': 'Miszna (hebr.\xa0\u200fמשנה\u200e miszna „nauczać”, „ustnie przekazywać”, „studiować”, „badać”, od \u200fשנה\u200e szana „powtarzać”, „różnić się”, „być odmiennym”; jid. Miszne) – w judaizmie uporządkowany zbiór tekstów ustnego prawa uzupełniający Torę (Prawo pisane). Według wierzeń judaizmu stanowi ustną, niespisaną część prawa nadanego przez Boga na Synaju, tzw. Torę ustną. Jest świętym tekstem judaizmu i jest traktowana na równi z Tanach (Biblią hebrajską). Zbiór był w Izraelu od wieków przekazywany ustnie z pokolenia na pokolenie, zwiększył swój rozmiar szczególnie w okresie od III w. p.n.e. do II w. n.e. w wyniku systematycznego uzupełniania komentarzy przez tannaitów, żydowskich nauczycieli prawa ustnego. Miszna została spisana dopiero w II–III w. Prace redakcyjne zapoczątkował rabin Akiba ben Josef, a kształt ostatecznej redakcji tekstu nadał Juda ha-Nasi. Miszna składa się z 6 porządków (hebr.: sedarim), które dzielą się na 63 trakt

In [5]:
print(len(train_dataset_json['data']))
train_dataset_json['data'][1]

8553


{'id': 8420,
 'title': 'Pomilio PD/PE',
 'summary': 'Pomilio PD/PE – rodzina włoskich samolotów rozpoznawczych z okresu I wojny światowej. Modele PD i PE różniły się silnikiem i wyglądem, zbudowano ich łącznie 1415 sztuk. Oba typy używane były od 1917 roku przez lotnictwo włoskie, będąc podstawowymi włoskimi samolotami rozpoznawczymi pod koniec wojny. Dysponowały dobrymi osiągami, lecz sprawiały pewne trudności w pilotażu i wycofano je z użytku w 1920 roku.',
 'url': 'https://pl.wikipedia.org/wiki/Pomilio_PD/PE',
 'paragraphs': [{'context': 'Samoloty Pomilio PD weszły na wyposażenie jednostek bojowych i zarazem do walki od lipca 1917 roku. Pierwsze otrzymały je w tym miesiącu eskadry (Squadriglia) nr 131 i 132, gdzie zastąpiły Pomilio PC. Do działań wchodziły następnie kolejne nowo formowane eskadry Pomilio: 133 (we wrześniu), 134 (w październiku), 135 (w styczniu 1918 roku), 136 (w maju 1918 roku). Eskadra 139, sformowana z dwóch sekcji początkowo przydzielonych do lotnictwa Marynarki

In [6]:
def prepare_dataset(dataset_dict):
    base_prompt = "kontekst: {context} pytanie: {question}"
    questions_with_context = []
    abstractive_answers = []

    for sample in dataset_dict:
        for paragraph in sample.get('paragraphs', []):
            curr_context = paragraph['context']
            for question in paragraph.get('qas', []):
                curr_question = question['question']
                if not question['is_impossible']: # ignore questions marked as impossible
                    for answer in question.get('answers', []):  
                        curr_answer = answer['generative_answer']
                        curr_prompt = base_prompt.format(context=curr_context, question=curr_question)
                        questions_with_context.append(curr_prompt)
                        abstractive_answers.append(curr_answer)

    dataset = {
        'question_and_context': questions_with_context,
        'abstractive_answer': abstractive_answers,
    }

    return dataset

In [7]:
train_dataset_dct = prepare_dataset(train_dataset_json['data'])
valid_dataset_dct = prepare_dataset(valid_dataset_json['data'])

In [8]:
print(f"Train dataset size: {len(train_dataset_dct['abstractive_answer'])}")
print(f"Valid dataset size: {len(valid_dataset_dct['abstractive_answer'])}")

Train dataset size: 46187
Valid dataset size: 5764


In [9]:
def print_sample(idx):
    print(valid_dataset_dct['question_and_context'][idx], len(valid_dataset_dct['question_and_context'][idx].split(' ')))
    print(valid_dataset_dct['abstractive_answer'][idx], len(valid_dataset_dct['abstractive_answer'][idx].split(' ')))
    print('')

print_sample(0)
print_sample(512)
print_sample(1234)
print_sample(2500)
print_sample(3000)
print_sample(-1)

kontekst: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik. pytanie: Czym są pisma rabiniczne? 86
kompilacją poglądów różnych rabinów na określony temat 7

kontekst: Akcja powieści rozgrywa się na przełomie XIX i XX wieku w Budapeszcie, stolicy Węgier. Oś fabuły stanowią walki o plac pomiędzy dwiema grupami chłopców. Grupa broniąca swego prawa do placu to Chłopcy z Placu Broni (w oryginale i innych językach nosi tytuł Chłopcy z ulicy Pawła), natomiast ich przeciwnicy, chcący zająć teren, by grać na nim 

In [10]:
# hard to say what should be maximum length of the question and answer (hope 512 and 64 will be more than enough)

In [11]:
from datasets import Dataset

valid_dataset = Dataset.from_dict(valid_dataset_dct)
train_dataset = Dataset.from_dict(train_dataset_dct)

/home/mklimek/Studies/NLP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
train_dataset

Dataset({
    features: ['question_and_context', 'abstractive_answer'],
    num_rows: 46187
})

In [13]:
valid_dataset

Dataset({
    features: ['question_and_context', 'abstractive_answer'],
    num_rows: 5764
})

In [14]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("allegro/plt5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
def tokenize_function(dataset_dct):
    model_inputs = tokenizer(dataset_dct['question_and_context'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(dataset_dct['abstractive_answer'], max_length=64, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)



ap: 100%|█████████████████████████| 5764/5764 [00:03<00:00, 1770.69 examples/s]

In [16]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("allegro/plt5-base")

In [17]:
from datasets import DatasetDict

dataset = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
})

In [18]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./lab9_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./lab9_logs",
    logging_steps=10,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    processing_class=tokenizer,
)

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.090000,0.063348
2,0.082300,0.055278
3,0.061100,0.053602


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=11547, training_loss=0.9470984451185628, metrics={'train_runtime': 18781.7515, 'train_samples_per_second': 7.377, 'train_steps_per_second': 0.615, 'total_flos': 1.0073882643451085e+17, 'train_loss': 0.9470984451185628, 'epoch': 3.0})

In [21]:
question = "Kto został mistrzem olimpijskim na skoczni normalnej?"
context = "Niepowodzeniem zakończył się start Małysza w zimowych igrzyskach olimpijskich w Turynie.\
W pierwszym konkursie na normalnej skoczni zajął siódme miejsce, chociaż mistrz olimpijski, Lars Bystol, \
w dwóch seriach pokonał Małysza zaledwie o metr. "

test_prompt = f"kontekst: {context} pytanie: {question}"
inputs = tokenizer(test_prompt, return_tensors="pt", max_length=512, truncation=True)
inputs = {key: value.to('cuda') for key, value in inputs.items()}
outputs = model.generate(inputs['input_ids'], max_length=64, num_beams=16, early_stopping=True)
print(outputs)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Odpowiedź:", answer)

tensor([[    0, 19253,  3826, 17615,     1]], device='cuda:0')
Odpowiedź: Lars Bystol


In [22]:
import evaluate

exact_match = evaluate.load("evaluate-metric/exact_match")

predictions = ["answer0 answer1", "answer2"]
references = ["answer0 answer1", "answer3 answer2 answer1"]

results = exact_match.compute(predictions=predictions, references=references)
print(results)

{'exact_match': np.float64(0.5)}


In [26]:
from sklearn.metrics import f1_score

def calc_mean_f1_score(predictions, references):
    MAX_L = 64
    results = []
    predictions_tokens = [tokenizer(prediction, max_length=MAX_L, truncation=True,)['input_ids'] for prediction in predictions]
    references_tokens = [tokenizer(ref, max_length=MAX_L, truncation=True)['input_ids'] for ref in references]
    for pred_t, ref_t in zip(predictions_tokens, references_tokens):
        min_len = min(len(pred_t), len(ref_t)) # let's make such assumption in calculating f1_score
        result = f1_score(pred_t[:min_len], ref_t[:min_len], average='micro')
        results.append(result)
    return sum(results)/len(results)

def calc_metrics(predictions, references):
    f1 = calc_mean_f1_score(predictions, references)
    em = float(exact_match.compute(predictions=predictions, references=references)['exact_match'])
    return(f1, em)

calc_metrics(predictions, references)

(0.8, 0.5)

In [24]:
valid_dataset_dct['question_and_context'][:5]

['kontekst: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik. pytanie: Czym są pisma rabiniczne?',
 'kontekst: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnyc

In [46]:
from tqdm import tqdm

def evaluate_on_dataset(dataset_dct, debug=False, limit=None, step=None):
    predicted_answers = []
    reference_answers = []
    for question, answer in tqdm(zip(dataset_dct['question_and_context'][:limit:step], 
                                 dataset_dct['abstractive_answer'][:limit:step]), 
                                 total=len(dataset_dct['question_and_context'][:limit:step])): 
        
        inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)
        inputs = {key: value.to('cuda') for key, value in inputs.items()}
        outputs = model.generate(inputs['input_ids'], max_length=64, num_beams=16, early_stopping=True)
        predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
        # converting answer to string for easier debugging and avoiding padding [0, ...] tokens at beginning of model response
        predicted_answers.append(predicted_answer)
        reference_answers.append(answer) 

        if debug:
            print(f"PROMPT   : {question}\n")
            print(f"PREDICTED: {predicted_answer}")
            print(f"TRUE     : {answer}\n\n")

    return calc_metrics(predicted_answers, reference_answers)
    
f1, em = evaluate_on_dataset(valid_dataset_dct, limit=100, step=5, debug=True)

 10%|████▍                                       | 2/20 [00:00<00:02,  6.61it/s]

PROMPT   : kontekst: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik. pytanie: Czym są pisma rabiniczne?

PREDICTED: kompilacją poglądów różnych rabinów na określony temat
TRUE     : kompilacją poglądów różnych rabinów na określony temat


PROMPT   : kontekst: Sformowany przez nią oddział partyzancki liczył 280 strzelców, kilkuset chłopów kosynierów i 60 kawalerzystów. Wraz z oddziałem rozpoczęła marsz w kierunku Dyneburga, po drodze do oddziału dołączali wciąż ochotnicy. 29 marca 1831 wkroczyła do Dus

 15%|██████▌                                     | 3/20 [00:00<00:02,  7.81it/s]

PROMPT   : kontekst: Zaręczyny pary ogłoszono 28 lipca 2007 za pośrednictwem biura Pałacu Buckingham. Kelly, ochrzczona w Kościele katolickim, dokonała konwersji na anglikanizm, aby jej przyszły mąż mógł utrzymać miejsce w linii sukcesji brytyjskiego tronu, w tym czasie bowiem Act of Settlement odsuwał od dziedziczenia tronu osoby, które poślubiły katolików. Ceremonia zaślubin miała miejsce 30 kwietnia 2008 w Kaplicy Świętego Jerzego na Zamku Windsor, a przewodniczył jej dziekan kapituły Windsoru, David Conner. Phillips jest pierwszym wnukiem królowej, który wstąpił w związek małżeński. Uroczystość relacjonował w specjalnym wydaniu magazyn Hello!, za co para otrzymała pięćset tysięcy funtów. Małżonkowie przejechali przez teren posiadłości bryczką, a następnie uczestniczyli w przyjęciu weselnym we Frogmore Cottage. Była to pierwsza okazja, w czasie której Katarzyna Middleton (późniejsza księżna Cambridge) reprezentowała samodzielnie swojego ówczesnego partnera, księcia Wilhelma, w oficj

 20%|████████▊                                   | 4/20 [00:00<00:02,  5.79it/s]

...

PROMPT   : kontekst: Odkrycia kolejnych transneptunowych obiektów pokazały, że w rzeczywistości Pas Kuipera nie jest źródłem komet krótkookresowych. Pochodzą one z tzw. dysku rozproszonego. Powstał on w początkowym okresie formowania się Układu Słonecznego, gdy Neptun oddalił się w rejony powstającego Pasa Kuipera, znajdującego się wtedy znacznie bliżej Słońca. Jego grawitacja zaburzyła orbity części obiektów z Pasa, rozciągając je i sprawiając, że obecnie ich peryhelia znajdują się w pobliżu orbity Neptuna. Dzięki temu mogą one, w przeciwieństwie do większości obiektów z Pasa, być wciąż wytrącane z orbit przez Neptuna i trafiać do wewnętrznego Układu. Ta grupa obiektów, nazwana dyskiem rozproszonym, jest uważana za źródło większości komet krótkookresowych. pytanie: Odkrycia jakich obiektów wpłynęły na poszerzenie wiedzy o Pasie Kurpiera?

PREDICTED: kolejnych transneptunowych obiektów
TRUE     : transneptunowych


PROMPT   : kontekst: W latach 1972–1981 był pracownikiem naukowo-dydakt

 90%|██████████████████████████████████████▋    | 18/20 [00:03<00:00,  4.50it/s]

PROMPT   : kontekst: Z drugiej jednak strony władze chińskie podjęły przed igrzyskami starania, by zapobiec ewentualnym protestom. Aktywista praw człowieka Ye Guozhu otrzymał czteroletni wyrok za uczestnictwo w organizowaniu demonstracji, skierowanej przeciw przymusowymi eksmisjom w Pekinie. Podczas przygotowań do organizacji igrzysk władze chińskie dokonały licznych przesiedleń, mających na celu przygotowania miejsca na przyszłe obiekty olimpijskie. Podczas samych Igrzysk władze chińskie również nie dopuszczały do przeprowadzania demonstracji przeciwko łamaniu praw człowieka. Mimo wyznaczenia trzech miejskich parków, w których mogłyby być organizowane demonstracje, władze odrzuciły wszystkie petycje w sprawie zorganizowania tam manifestacji. pytanie: Za co ukarano Guozhu?

PREDICTED: uczestnictwo w organizowaniu demonstracji, skierowanej przeciw przymusowymi eksmisjom w Pekinie
TRUE     : za uczestnictwo w organizowaniu demonstracji


PROMPT   : kontekst: Szybko dostał angaż do kilku 

100%|███████████████████████████████████████████| 20/20 [00:04<00:00,  4.90it/s]

PROMPT   : kontekst: Ze względu na opór stawiany przez Forstera Himmler w dniu 26 października 1939 podjął decyzję o chwilowym wstrzymaniu wysiedleń z ziem wcielonych do Rzeszy. Już cztery dni później wydał jednak rozporządzenie nr 1/II do dekretu Hitlera o umocnieniu Niemczyzny, w którym nakazywał, aby do końca lutego 1940 z terenów Okręgu Rzeszy Gdańsk-Prusy Zachodnie wysiedleni zostali wszyscy Żydzi oraz Polacy przybyli na Pomorze po 1918 roku. 3 listopada 1939 ponownie wezwał natomiast do kontynuowania na Pomorzu i w Poznańskiem akcji osadniczo-wysiedleńczej związanej z ewakuacją Niemców bałtyckich. Z kolei w dniu 15 listopada Hildebrandt zorganizował w Gdańsku naradę z udziałem przedstawicieli władz okręgu, której celem było wypracowanie wspólnego stanowiska w sprawie planowanych przesiedleń. W trakcie konferencji najwięcej miejsca poświęcono kwestii osadnictwa Niemców bałtyckich. Hildebrandt wychodząc nieco naprzeciw postulatom Forstera, zgodził się, że Okręg Gdańsk-Prusy Zachodn

In [47]:
f1, em = evaluate_on_dataset(valid_dataset_dct)


00%|███████████████████████████████████████| 5764/5764 [15:10<00:00,  6.33it/s]

In [50]:
print('VALIDATION DATASET')
print(f"f1 score    = {f1}")
print(f"exact match = {em}")

VALIDATION DATASET
f1 score    = 0.6451048679009687
exact match = 0.5293199167244969


In [51]:
trainer.save_model("./lab9_model")
tokenizer.save_pretrained("./lab9_model")

('./lab9_model/tokenizer_config.json',
 './lab9_model/special_tokens_map.json',
 './lab9_model/spiece.model',
 './lab9_model/added_tokens.json')

## Komentarz:

Na zbiorze walidacyjnym model wydaje się dawać zadowalające wyniki. Być może metryki nie pokazują w pełni jego działania - zarówno exact_match jak i f_1 score uwzględniają tylko bezpośrednie porównanie wyrazów/tokenów z oczekiwaną odpowiedzą, nie patrząc na to, czy udzielone przez model odpowiedzi są zgodne z kontekstem.

Na przykład w pytaniu:

```
Za co ukarano Guozhu?
PREDICTED: uczestnictwo w organizowaniu demonstracji, skierowanej przeciw przymusowymi eksmisjom w Pekinie
TRUE     : za uczestnictwo w organizowaniu demonstracji
```
model udzielił prawidłowej odpowiedzi, w dobrej formie, dodając dodatkowe szczegóły w porównaniu z zakładaną odpowiedzią, jednak według obydwu używanych metryk odpowiedź modelu będzie uznana za całkowicie złą.

Model ma też problem z dobraniem odpowiedniej formy odpowiedzi, podając ją zazwyczaj jako bezpośredni cytat z tekstu, na przykład:
```
[...] W latach 2003–2006 był dziekanem Wydziału Prawa Europejskiego Instytutu Uniwersyteckiego. [...] W jakiej jednostce uczelni prawnik obejmował stanowisko dziekana?
PREDICTED: Wydziału Prawa Europejskiego Instytutu Uniwersyteckiego
TRUE     : na Wydziale Prawa Europejskiego Instytutu Uniwersyteckiego
```

Możliwe, że w zbiorze treningowym był zbyt mały udział odpowiedzi w innej formie niż ta, która występowała w kontekście, aby model nauczył się też do dopasowywania odmiany odpowiedzi do formy pytania. 

In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("allegro/plt5-base")
model = T5ForConditionalGeneration.from_pretrained("./lab9_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [23]:
question = "Jakie urządzenia do końca 2024 roku mają mieć USB typu C?"
context = "Do końca 2024 roku wszystkie telefony komórkowe, tablety i aparaty fotograficzne sprzedawane w Unii Europejskiej \
będą musiały być wyposażone w port USB typu C. Wiosną 2026 roku Unia obejmie tym wymogiem także laptopy. \
Parlament przyjął nowe przepisy na wtorkowym posiedzeniu plenarnym. \
602 posłów głosowało za, 13– przeciw, zaś 8 posłów wstrzymało się od głosu.  "

test_prompt = f"kontekst: {context} pytanie: {question}"
inputs = tokenizer(test_prompt, return_tensors="pt", max_length=512, truncation=True)
inputs = {key: value.to('cuda') for key, value in inputs.items()}
outputs = model.generate(inputs['input_ids'], max_length=64, num_beams=16, early_stopping=True)
print(outputs)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Odpowiedź:", answer)

tensor([[    0, 15344, 31897,   261, 27705,   273,   264,  6747,   273, 18172,
         22989,   260,  1211,  1884,     1]], device='cuda:0')
Odpowiedź: telefony komórkowe, tablety i aparaty fotograficzne sprzedawane w Unii Europejskiej


In [24]:
question = "Czy w 2024 roku laptopy mają być wyposażone w USB typu C?"
test_prompt = f"kontekst: {context} pytanie: {question}"
inputs = tokenizer(test_prompt, return_tensors="pt", max_length=512, truncation=True)
inputs = {key: value.to('cuda') for key, value in inputs.items()}
outputs = model.generate(inputs['input_ids'], max_length=64, num_beams=16, early_stopping=True)
print(outputs)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Odpowiedź:", answer)

tensor([[  0, 436,   1]], device='cuda:0')
Odpowiedź: tak


In [25]:
import pandas as pd

df = pd.read_csv("./simple-legal-questions-pl-main/questions.csv")
df = df[df['has_answer'] == True]
df

,passage_id,question,passage,has_answer,year,position,art
0,1997_553_345.txt,"Czy żołnierz, który dopuszcza się czynnej napa...","Art. 345. § 1. Żołnierz, który dopuszcza sie...",True,1997,553,345
1,2004_177_21.txt,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej pow...,True,2004,177,21
2,1996_465_111.txt,Do jakiej wysokości za zobowiązania spółki odp...,Art. 111. Komandytariusz odpowiada za zobowią...,True,1996,465,111
3,1994_591_35.txt,"Kiedy ustala się wartość majątku obrotowego, k...",Art. 35. 1. Wartość rzeczowych składników m...,True,1994,591,35
4,2001_1441_74.txt,"Jakiej karze podlega armator, który wykonuje r...","Art. 74. 1. Armator, który wykonuje rybołóws...",True,2001,1441,74
...,...,...,...,...,...,...,...
1470,1995_479.txt-Art. 29.,Jakim przepisom podlegają przychody kościelnyc...,1. Majątek i przychody kościelnych osób prawny...,True,1995,479,29
1471,1995_482.txt-Art. 27.,Jakim przepisom podlegają przychody kościelnyc...,1. Majątek i przychody kościelnych osób prawny...,True,1995,482,27
1472,1997_554.txt-Art. 19.,Jakim przepisom podlegają przychody kościelnyc...,1. Majątek i przychody kościelnych osób prawny...,True,1997,554,19
1473,1995_481.txt-Art. 28.,Jakim przepisom podlegają przychody kościelnyc...,1. Majątek i przychody Kościoła oraz jego osób...,True,1995,481,28


In [26]:
questions_df = pd.read_json("./simple-legal-questions-pl-main/questions.jl", lines=True)
questions_df

,_id,text
0,1,"Czy żołnierz, który dopuszcza się czynnej napa..."
1,2,Z ilu osób składa się komisja przetargowa?
2,3,Do jakiej wysokości za zobowiązania spółki odp...
3,4,"Kiedy ustala się wartość majątku obrotowego, k..."
4,5,"Jakiej karze podlega armator, który wykonuje r..."
...,...,...
1431,1432,Jakim przepisom podlegają przychody kościelnyc...
1432,1433,Jakim przepisom podlegają przychody kościelnyc...
1433,1434,Jakim przepisom podlegają przychody kościelnyc...
1434,1435,Jakim przepisom podlegają przychody kościelnyc...


In [27]:
passages_df = pd.read_json("./simple-legal-questions-pl-main/passages.jl", lines=True)
passages_df

,_id,title,text
0,2004_2387_1,Ustawa z dnia 27 sierpnia 2004 r. o ratyfikacj...,Art. 1. Wyraża się zgodę na dokonanie przez Pr...
1,2004_2387_2,Ustawa z dnia 27 sierpnia 2004 r. o ratyfikacj...,Art. 2. Ustawa wchodzi w życie po upływie 14 d...
2,2001_1209_1,Ustawa z dnia 26 lipca 2001 r. o nabywaniu prz...,Art. 1. 1. Osobom fizycznym będącym w dniu 26 ...
3,2001_1209_2,Ustawa z dnia 26 lipca 2001 r. o nabywaniu prz...,"Art. 2. 1. Do osób, o których mowa w art. 1 us..."
4,2001_1209_3,Ustawa z dnia 26 lipca 2001 r. o nabywaniu prz...,"Art. 3. 1. Do osób, które skorzystały z uwłasz..."
...,...,...,...
26282,1999_934_2,Ustawa z dnia 22 lipca 1999 r. o zmianie ustaw...,Art. 2. Ustawa wchodzi w życie po upływie 14 d...
26283,2001_808_1,Ustawa z dnia 21 czerwca 2001 r. o zmianie ust...,Art. 1. W ustawie z dnia 18 grudnia 1998 r. o ...
26284,2001_808_2,Ustawa z dnia 21 czerwca 2001 r. o zmianie ust...,Art. 2. W stosunku do umów o kredyt na realiza...
26285,2001_808_3,Ustawa z dnia 21 czerwca 2001 r. o zmianie ust...,Art. 3. Dotychczasowe przepisy wykonawcze wyda...


In [28]:
answers_df = pd.read_json("./simple-legal-questions-pl-main/answers.jl", lines=True)
answers_df = answers_df[answers_df['score'] > 0]
answers_df

,score,question-id,answer
0,1.0,1,"Tak, podlega karze aresztu wojskowego albo poz..."
1,1.0,2,Komisja przetargowa składa się z co najmniej t...
2,1.0,3,Komandytariusz odpowiada za zobowiązania spółk...
3,1.0,4,Wartość rzeczowych składników majątku obrotowe...
4,1.0,5,Podlega karze pieniężnej do wysokości 1 000 00...
...,...,...,...
675,1.0,1130,związek rewizyjny
676,1.0,1131,Prezes Urzędu Ochrony Konkurencji i Konsumentów
677,1.0,1132,minister właściwy do spraw rolnictwa
678,1.0,1133,1) w obrocie giełdowym - z chwilą zapłacenia c...


In [29]:
relevant_df = pd.read_json("./simple-legal-questions-pl-main/relevant.jl", lines=True)
relevant_df

,question-id,passage-id,score
0,1,1997_553_345,1
1,2,2004_177_21,1
2,3,1996_465_111,1
3,4,1994_591_35,1
4,5,2001_1441_74,1
...,...,...,...
1431,1432,1995_479_29,1
1432,1433,1995_482_27,1
1433,1434,1997_554_19,1
1434,1435,1995_481_28,1


In [30]:
ans_quests_df = answers_df.merge(questions_df, left_on='question-id', right_on='_id', how='left')
ans_quests_rel_df = ans_quests_df.merge(relevant_df[['question-id', 'passage-id']], left_on='question-id', right_on='question-id', how='left')
ans_quests_rel_df = ans_quests_rel_df.rename(columns={'text': 'question'})
ans_quests_rel_text_df = ans_quests_rel_df.merge(passages_df[['_id', 'text']], left_on='passage-id', right_on='_id', how='left')
ans_quests_rel_text_df = ans_quests_rel_text_df.drop(columns=['score', 'question-id', '_id_x', 'passage-id', '_id_y'])
ans_quests_rel_text_df

,answer,question,text
0,"Tak, podlega karze aresztu wojskowego albo poz...","Czy żołnierz, który dopuszcza się czynnej napa...","Art. 345. § 1. Żołnierz, który dopuszcza się c..."
1,Komisja przetargowa składa się z co najmniej t...,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej powo...
2,Komandytariusz odpowiada za zobowiązania spółk...,Do jakiej wysokości za zobowiązania spółki odp...,Art. 111. Komandytariusz odpowiada za zobowiąz...
3,Wartość rzeczowych składników majątku obrotowe...,"Kiedy ustala się wartość majątku obrotowego, k...",Art. 35. 1. Wartość rzeczowych składników mają...
4,Podlega karze pieniężnej do wysokości 1 000 00...,"Jakiej karze podlega armator, który wykonuje r...","Art. 74. 1. Armator, który wykonuje rybołówstw..."
...,...,...,...
571,związek rewizyjny,Jaki związek tworzą banki spółdzielcze?,Art. 35. 1. Banki spółdzielcze tworzą związek ...
572,Prezes Urzędu Ochrony Konkurencji i Konsumentów,Kto wydaje decyzję o uznaniu praktyki za ogran...,Art. 9. Prezes Urzędu Ochrony Konkurencji i Ko...
573,minister właściwy do spraw rolnictwa,Kto powołuje okręgowych inspektorów rybołówstw...,Art. 51. 1. Okręgowi inspektorzy rybołówstwa m...
574,1) w obrocie giełdowym - z chwilą zapłacenia c...,W jakim momencie powstaje obowiązek podatkowy?,Art. 4. Obowiązek podatkowy powstaje: 1) w obr...


In [31]:
size = ans_quests_rel_text_df.shape[0]
size

576

In [32]:
ans_quests_rel_text_df['question_and_context'] = pd.Series(['kontekst: ' for _ in range(size)]) \
                                                + ans_quests_rel_text_df['text'] \
                                                + pd.Series([' pytanie: ' for _ in range(size)])\
                                                + ans_quests_rel_text_df['question']

ans_quests_rel_text_df = ans_quests_rel_text_df.rename(columns={'answer': 'abstractive_answer'})
ans_quests_rel_text_df['abstractive_answer'] = ans_quests_rel_text_df['abstractive_answer'].str.lower()
ans_quests_rel_text_df = ans_quests_rel_text_df.drop(columns=['text', 'question'])
ans_quests_rel_text_df

,abstractive_answer,question_and_context
0,"tak, podlega karze aresztu wojskowego albo poz...","kontekst: Art. 345. § 1. Żołnierz, który dopus..."
1,komisja przetargowa składa się z co najmniej t...,kontekst: Art. 21. 1. Członków komisji przetar...
2,komandytariusz odpowiada za zobowiązania spółk...,kontekst: Art. 111. Komandytariusz odpowiada z...
3,wartość rzeczowych składników majątku obrotowe...,kontekst: Art. 35. 1. Wartość rzeczowych skład...
4,podlega karze pieniężnej do wysokości 1 000 00...,"kontekst: Art. 74. 1. Armator, który wykonuje ..."
...,...,...
571,związek rewizyjny,kontekst: Art. 35. 1. Banki spółdzielcze tworz...
572,prezes urzędu ochrony konkurencji i konsumentów,kontekst: Art. 9. Prezes Urzędu Ochrony Konkur...
573,minister właściwy do spraw rolnictwa,kontekst: Art. 51. 1. Okręgowi inspektorzy ryb...
574,1) w obrocie giełdowym - z chwilą zapłacenia c...,kontekst: Art. 4. Obowiązek podatkowy powstaje...


In [33]:
print(ans_quests_rel_text_df.loc[0]['question_and_context'])
print('')
print(ans_quests_rel_text_df.loc[0]['abstractive_answer'])

kontekst: Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3. § 2. Jeżeli sprawca dopuszcza się czynnej napaści w związku z pełnieniem przez przełożonego obowiązków służbowych albo wspólnie z innymi żołnierzami lub w obecności zebranych żołnierzy, podlega karze pozbawienia wolności od 6 miesięcy do lat 8. § 3. Jeżeli sprawca czynu określonego w § 1 lub 2 używa broni, noża lub innego podobnie niebezpiecznego przedmiotu, podlega karze pozbawienia wolności od roku do lat 10. § 4. Karze przewidzianej w § 3 podlega sprawca czynu określonego w § 1 lub 2, jeżeli jego następstwem jest skutek określony w art. 156 lub 157 § 1. pytanie: Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?

tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.


In [40]:
import evaluate
from tqdm import tqdm
from sklearn.metrics import f1_score
# same as previous but in one place

exact_match = evaluate.load("evaluate-metric/exact_match")

def calc_mean_f1_score(predictions, references):
    MAX_L = 64
    results = []
    predictions_tokens = [tokenizer(prediction, max_length=MAX_L, truncation=True,)['input_ids'] for prediction in predictions]
    references_tokens = [tokenizer(ref, max_length=MAX_L, truncation=True)['input_ids'] for ref in references]
    for pred_t, ref_t in zip(predictions_tokens, references_tokens):
        min_len = min(len(pred_t), len(ref_t)) # let's make such assumption in calculating f1_score
        result = f1_score(pred_t[:min_len], ref_t[:min_len], average='micro')
        results.append(result)
    return sum(results)/len(results)

def calc_metrics(predictions, references):
    f1 = calc_mean_f1_score(predictions, references)
    em = float(exact_match.compute(predictions=predictions, references=references)['exact_match'])
    return(f1, em)
    
def evaluate_on_dataset(dataset_dct, debug=False, limit=None, step=None):
    predicted_answers = []
    reference_answers = []
    for question, answer in tqdm(zip(dataset_dct['question_and_context'][:limit:step], 
                                 dataset_dct['abstractive_answer'][:limit:step]), 
                                 total=len(dataset_dct['question_and_context'][:limit:step])): 
        
        inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)
        inputs = {key: value.to('cuda') for key, value in inputs.items()}
        outputs = model.generate(inputs['input_ids'], max_length=64, num_beams=16, early_stopping=True)
        predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
        # converting answer to string for easier debugging and avoiding padding [0, ...] tokens at beginning of model response
        predicted_answers.append(predicted_answer)
        reference_answers.append(answer) 

        if debug:
            print(f"PROMPT   : {question}\n")
            print(f"PREDICTED: {predicted_answer}")
            print(f"TRUE     : {answer}\n\n")

    return calc_metrics(predicted_answers, reference_answers)

In [41]:
test_dataset_dct = ans_quests_rel_text_df.to_dict(orient='list')
f1, em = evaluate_on_dataset(test_dataset_dct, limit=50, step=5, debug=True)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

PROMPT   : kontekst: Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3. § 2. Jeżeli sprawca dopuszcza się czynnej napaści w związku z pełnieniem przez przełożonego obowiązków służbowych albo wspólnie z innymi żołnierzami lub w obecności zebranych żołnierzy, podlega karze pozbawienia wolności od 6 miesięcy do lat 8. § 3. Jeżeli sprawca czynu określonego w § 1 lub 2 używa broni, noża lub innego podobnie niebezpiecznego przedmiotu, podlega karze pozbawienia wolności od roku do lat 10. § 4. Karze przewidzianej w § 3 podlega sprawca czynu określonego w § 1 lub 2, jeżeli jego następstwem jest skutek określony w art. 156 lub 157 § 1. pytanie: Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?

PREDICTED: tak
TRUE     : tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.




 30%|█████████████▏                              | 3/10 [00:01<00:02,  3.04it/s]

PROMPT   : kontekst: Art. 31. 1. Zwalnia się od akcyzy, jeżeli wynika to z porozumień międzynarodowych, lub zasady wzajemności, czynności podlegające opodatkowaniu akcyzą wobec instytucji Wspólnot Europejskich oraz wobec: organizacji międzynarodowych, przedstawicielstw dyplomatycznych, urzędów konsularnych oraz członków personelu tych przedstawicielstw i urzędów, a także innych osób zrównanych z nimi na podstawie ustaw, umów lub zwyczajów międzynarodowych, jeżeli nie są obywatelami polskimi i nie mają stałego miejsca pobytu na terytorium kraju. 2. Zwalnia się od akcyzy, jeżeli wynika to z porozumień międzynarodowych, czynności podlegające opodatkowaniu akcyzą, których przedmiotem są wyroby akcyzowe przeznaczone dla sił zbrojnych Państw-Stron Traktatu Północnoatlantyckiego, sił zbrojnych uczestniczących w Partnerstwie dla Pokoju oraz dla Kwatery Głównej Wielonarodowego Korpusu Północno-Wschodniego i członków jej personelu oraz dowództw sojuszniczych, w szczególności Centrum Szkolenia Si

 40%|█████████████████▌                          | 4/10 [00:01<00:02,  2.41it/s]

...

 80%|███████████████████████████████████▏        | 8/10 [00:03<00:00,  2.38it/s]

PROMPT   : kontekst: Art. 36. 1. Armatorowi, z którego statku podczas przebywania w polskich obszarach morskich dokonano, z naruszeniem przepisów ustawy, zanieczyszczenia środowiska morskiego, w związku z eksploatacją statku lub zatopieniem w morzu odpadów i innych substancji - wymierza się karę pieniężną o równowartości do 1 000 000 jednostek obliczeniowych, zwanych Specjalnym Prawem Ciągnienia (SDR), określanych przez Międzynarodowy Fundusz Walutowy. 2. Karze, o której mowa w ust. 1, podlega armator statku o polskiej przynależności, z którego dokonano, z naruszeniem przepisów ustawy, zanieczyszczenia środowiska morskiego poza polskimi obszarami morskimi, w związku z eksploatacją statku lub zatopieniem w morzu odpadów albo innych substancji, bez wymaganego zezwolenia lub wbrew jego warunkom. pytanie: Jakiej karze podlega armator, który wykonuje rybołówstwo morskie w polskich obszarach morskich, z naruszeniem przepisów ustawy?

PREDICTED: pieniężnej o równowartości do 1 000 000 jedno

 90%|███████████████████████████████████████▌    | 9/10 [00:03<00:00,  2.09it/s]

PROMPT   : kontekst: Art. 281. 1. Przewodniczącego kolegium i członków składu kolegium wyznacza Przewodniczący Wydziału Spraw Spornych. 2. W skład kolegium oprócz przewodniczącego kolegium wchodzą jako jego członkowie: jeden ekspert wyznaczony z listy ustalonej przez Prezesa Urzędu Patentowego oraz jeden przedstawiciel organizacji, o których mowa w art. 9, wyznaczony z listy przedstawicieli, uzgodnionej z Prezesem Urzędu Patentowego przez te organizacje. 3. Jeżeli wyznaczono kolegium pięcioosobowe, oprócz przewodniczącego kolegium w jego skład wchodzi po dwóch członków z każdej z list, o których mowa w ust. 2. 4. Przedstawicielem organizacji, o którym mowa w ust. 2, nie może być czynny zawodowo rzecznik patentowy. pytanie: Kto wchodzi w skład kolegium doradczego wojewody?

PREDICTED: jeden ekspert wyznaczony z listy ustalonej przez Prezesa Urzędu Patentowego oraz jeden przedstawiciel organizacji, o których mowa w art. 9, wyznaczony z listy przedstawicieli, uzgodnionej z Prezesem Urzędu


00%|███████████████████████████████████████████| 10/10 [00:03<00:00,  2.54it/s]

In [43]:
f1, em = evaluate_on_dataset(test_dataset_dct)


00%|█████████████████████████████████████████| 576/576 [01:55<00:00,  5.00it/s]

In [44]:
print('TEST DATASET (LEGAL QUESTIONS)')
print(f"f1 score    = {f1}")
print(f"exact match = {em}")

TEST DATASET (LEGAL QUESTIONS)
f1 score    = 0.35401868910318446
exact match = 0.19791666666666666


Wyniki ze zbioru walidacyjnego dla porównania:

```
VALIDATION DATASET
f1 score    = 0.6451048679009687
exact match = 0.5293199167244969
```

## Komentarz:
Jak widać wyniki uzyskane na zbiorze testowym są wyraźnie gorsze od wyników uzyskanych na zbiorze walidacyjny (29 punktów procentowych spadku w metryce f1 i 33 pp w dokładnych dopasowaniach). W wyświetlonych przykładach widać jednak, że model potrafi udzelić na pytanie poprawnej (np. dwa ostatnie pytania) lub zbliżonej do poprawnej odpowiedzi (np. pierwsze pytanie), która może jednak nie zostać zaliczona przez stosowane metryki.

Na podstawie wyświetlonych przykładów można wskazać potencjalne powody tak dużej różnicy:
 - w zbiorze pytań prawnych problemem może być duża objętość promptów - fragmenty są dłuższe niż w zbiorze treningowym i walidacyjnym, co dla niezbyt dużego modelu, może stanowić problem
 - inna forma kontekstu - w kotntekście więcej jest numerów, paragrafów, ustępów, etc. Używane jest też inne, bardziej skomolikowane, formalne słownictwo, które nie było zbyt dobrze reprezentowane w zbiorze treningowym
 - bardziej rozbudowane odpowiedzi - zbiór z pytaniami prawnymi zawiera często bardziej rozbudowane odpowiedzi, często z powtórzeniem pytania, tymczasem w zbiorze, na którym model był trenowany odpowiedzi były maksymalnie któtkie i precyzyjne. Np. w przykładzie poniżej odpowiedź przewidziana przez model może być uznana za prawidłową, podczas gdy odpowiedź oczekiwana według zbioru jest od niego znacznie dłuższa:
```
PROMPT   : kontekst: Art. 408. Po wysłuchaniu głosów stron sąd niezwłocznie przystępuje do narady. pytanie: Co robi sąd po wysłuchaniu głosów stron?

PREDICTED: przystępuje do narady
TRUE     : po wysłuchaniu głosów stron sąd niezwłocznie przystępuje do narady
```

Podobnie w przypadku poniższego pytania - odpowiedź `tak` jest poprawna i byłaby zapewne oczekiwana w zbiorze treningowym i walidacyjnym, tymczasem zbiór pytań prawnych wymaga dodatkowego rozwinięcie (o którym nie ma mowy w pytaniu)
```
kontekst: ... pytanie: Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?

PREDICTED: tak
TRUE     : tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.
```

**Dlaczego *extractive question answering*  nie sprawdza się dla języków z fleksją?**

Przy zastosowaniu ekstrakcyjnego odpowiadania na pytania odpowiedzi zachowują taką samą formę, w jakiej były zapisane w kontekście, przez co mogą być niedopasowane do pytania. Utrudnia to czytanie odpowiedzi przez ludzi i może uniemożliwić przetwarzanie odpowiedzi przez zautomatyzowane systemy (które mogą na przykład zliczać wystąpienia jakiejś odpowiedzi i oczekiwać zawsze jej podstawowej formy). Podstawowa forma extractive question answering nie jest też przystosowana do udzielania odpowiedzi w stylu tak/nie, które można uzyskać z modelu abstrakcyjnego.